In [1]:
import numpy as np
from scipy.integrate import solve_ivp, odeint
import deepdish as dd
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import h5py

def pendulum(t, thetas, g, l):
    theta, dot_theta = thetas # y0, y1
    dots = (dot_theta, -(g/l)*np.sin(theta))
    return  dots # y0_dot, y1_dot

def get_trajectory(timesteps, dt, angle, omega0, length, g):
    theta0 = np.radians(angle) # initial anglee

    tmin = 0.0
    tmax = timesteps*dt
    ts = np.linspace(tmin, tmax, timesteps)
    sol = solve_ivp(pendulum, [tmin, tmax], [theta0, omega0], t_eval = ts, args=(g,length))

    # save the x, y coordinated of the pendulum
    xy = np.zeros_like(sol.y)
    xy[0] = length*np.sin(sol.y[0])
    xy[1] = length*np.cos(sol.y[0])

    cartesian = xy.T
    phase_space = sol.y.T
    
    labels =  {'initial_angle': angle, 
               'initial_velocity': omega0, 
               'gravity': g, 
               'length': length}
    
    return cartesian, phase_space, labels

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp, odeint
import deepdish as dd
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import h5py

def pendulum(t, thetas, g, l):
    theta, dot_theta = thetas # y0, y1
    dots = (dot_theta, -(g/l)*np.sin(theta))
    return  dots # y0_dot, y1_dot

def get_trajectory(timesteps, dt, angle, omega0, length, g):
    theta0 = np.radians(angle) # initial anglee

    tmin = 0.0
    tmax = timesteps*dt
    ts = np.linspace(tmin, tmax, timesteps)
    sol = solve_ivp(pendulum, [tmin, tmax], [theta0, omega0], t_eval = ts, args=(g,length))

    # save the x, y coordinated of the pendulum
    xy = np.zeros_like(sol.y)
    xy[0] = length*np.sin(sol.y[0])
    xy[1] = length*np.cos(sol.y[0])

    cartesian = xy.T
    phase_space = sol.y.T
    
    labels =  {'initial_angle': angle, 
               'initial_velocity': omega0, 
               'gravity': g, 
               'length': length}
    
    return cartesian, phase_space, labels

def coords_to_pixels(world_size, x, y, res):
        """Maps coordinates from world space to pixel space
        Args: x,y (float): x,y coordinate of the world space.
            res (int): Image resolution in pixels (images are square).
        Returns: (int, int): Tuple of coordinates in pixel space.
        """
        pix_x = np.rint(res*(x + world_size)/(2*world_size))
        pix_y = np.rint(res*(y + world_size)/(2*world_size))
        return (int(pix_x), int(pix_y))


def draw(q, resolution, world_size, ball_size, ball_color, filter_size, resize=None):
    """Returns array of the environment evolution
       vid (np.ndarray): Rendered rollout as a sequence of images
    """    
    background_color = 0.0
    
    rollout_length = len(q)
    
    if resize is None:
        resize = resolution

    vid = np.zeros((rollout_length, resize, resize), dtype=np.float32)
    for t in range(rollout_length):
        temp = np.zeros((resolution, resolution))
        pixels = coords_to_pixels(world_size, q[t,0], q[t,1], resolution)
        temp = cv2.circle(temp, pixels, ball_size, ball_color, thickness=-1)
        temp = cv2.blur(temp, (filter_size, filter_size))
        temp = cv2.blur(temp, (filter_size, filter_size))
        if resize is not None:
            temp = cv2.resize(temp, (resize, resize))
        vid[t] = temp

    vid += background_color
    vid[vid > 1.] = 1.
    vid[vid < 0.] = 0.
    
    return vid

def create_dataset(fname, dt, timesteps, final_resolution, angle_range, velocity_range, g_range, length_range, g_exclude = None, length_exclude = None):
    world_size = 2.5
    resolution=512
    ball_size=int(resolution/8)
    filter_size=int(ball_size/8)
    ball_color=1
    params = {'dt': dt,
        'timesteps': timesteps,
        'world_size': world_size,
        'initial_resolution': resolution,
        'final_resolution': final_resolution, 
        'filter_size': filter_size, 
        'ball_size': ball_size,
        'color': ball_color,
        'angle_range': angle_range,
        'velocity_range': velocity_range,
        'g_range': g_range,
        'length_range': length_range
    }
    num_datapoints = angle_range[2] * velocity_range[2] * g_range[2] * length_range[2]

    fg = h5py.File(fname, 'w')

    for k in fg.keys():
        del fg[k]
    for k in fg.attrs.keys():
        del fg.attrs[k]

    # Set global attributes
    for k,v in params.items():
        fg.attrs[k] = v

    fg.create_dataset('frames',(num_datapoints, timesteps, final_resolution, final_resolution), 
                     chunks=(1, timesteps, final_resolution, final_resolution), compression='gzip')
    fg.create_dataset('phase_space',(num_datapoints, timesteps, 2), 
                     chunks=(1, timesteps, 2), compression='gzip')

    fg.create_group('labels')
    fg.create_dataset('labels/pendulum_length', (num_datapoints,1), chunks=True)
    fg.create_dataset('labels/g', (num_datapoints,1), chunks=True)
    fg.create_dataset('labels/initial_angle', (num_datapoints,1), chunks=True)
    fg.create_dataset('labels/initial_velocity', (num_datapoints,1), chunks=True)
    fg.keys()

    i=0
    print(f'Creating {num_datapoints} datapoints')
    
    angle_list = np.random.uniform(*angle_range[:-1],num_datapoints)
    velocity_list = np.random.uniform(*velocity_range[:-1],num_datapoints)
    g_list = np.random.uniform(*g_range[:-1],num_datapoints)
    length_list = np.random.uniform(*length_range[:-1],num_datapoints)

    if g_exclude != None or length_exclude != None:
        for idx in range(num_datapoints):
            while g_list[idx] >= g_exclude[0] and g_list[idx] <= g_exclude[1] and length_list[idx] >= length_exclude[0] and length_list[idx] <= length_exclude[1]:
                g_list[idx] = np.random.uniform(*g_range[:-1],1)[0]
                length_list[idx] = np.random.uniform(*length_range[:-1],1)[0]

    for angle,omega0,g,pendulum_length in zip(angle_list,velocity_list,g_list,length_list):        
        fg['labels']['pendulum_length'][i]  = pendulum_length
        fg['labels']['g'][i]                = g
        fg['labels']['initial_angle'][i]    = angle
        fg['labels']['initial_velocity'][i] = omega0

        i+=1
        if (i % 100) == 0:
            print(f'{i:04d}/{num_datapoints} finished')
    
    fg.close()

final_resolution = 64
dt = 0.01
timesteps = 1000

n_div = 6
angle_range = (30, 170, n_div)
velocity_range = (-2, 2, n_div)
g_range = (8, 12, n_div)
length_range = (1.2, 1.4, n_div)

g_exclude = (8, 12)
length_exclude = (1.2, 1.4)

num_datapoints = angle_range[2] * velocity_range[2] * g_range[2] * length_range[2]

ang_str = '-'.join([str(a) for a in angle_range[:2]])
vel_str = '-'.join([f'{a:.2f}' for a in velocity_range[:2]])
g_str = '-'.join([f'{a:.2f}' for a in g_range[:2]])
len_str = '-'.join([f'{a:.2f}' for a in length_range[:2]])

fname = f'../data/pixel_pendulum_n_{num_datapoints}_steps_{timesteps}_dt_{dt:.2f}'+\
        f'_angle_{ang_str}_vel_{vel_str}_len_{len_str}_g_{g_str}_val.hd5'

create_dataset(fname, dt, timesteps, final_resolution, angle_range, velocity_range, g_range, length_range)
print(fname)

n_div = 6
angle_range = (30, 170, n_div)
velocity_range = (-2, 2, n_div)
g_range = (8, 12, n_div)
length_range = (1.2, 1.4, n_div)

num_datapoints = angle_range[2] * velocity_range[2] * g_range[2] * length_range[2]

ang_str = '-'.join([str(a) for a in angle_range[:2]])
vel_str = '-'.join([f'{a:.2f}' for a in velocity_range[:2]])
g_str = '-'.join([f'{a:.2f}' for a in g_range[:2]])
len_str = '-'.join([f'{a:.2f}' for a in length_range[:2]])

fname = f'../data/pixel_pendulum_n_{num_datapoints}_steps_{timesteps}_dt_{dt:.2f}'+\
        f'_angle_{ang_str}_vel_{vel_str}_len_{len_str}_g_{g_str}_test.hd5'

create_dataset(fname, dt, timesteps, final_resolution, angle_range, velocity_range, g_range, length_range)
print(fname)


n_div = 6
angle_range = (30, 170, n_div)
velocity_range = (-2, 2, n_div)
g_range = (12, 12.5, n_div)
length_range = (1.4, 1.45, n_div)

num_datapoints = angle_range[2] * velocity_range[2] * g_range[2] * length_range[2]

ang_str = '-'.join([str(a) for a in angle_range[:2]])
vel_str = '-'.join([f'{a:.2f}' for a in velocity_range[:2]])
g_str = '-'.join([f'{a:.2f}' for a in g_range[:2]])
len_str = '-'.join([f'{a:.2f}' for a in length_range[:2]])

fname = f'../data/pixel_pendulum_n_{num_datapoints}_steps_{timesteps}_dt_{dt:.2f}'+\
        f'_angle_{ang_str}_vel_{vel_str}_len_{len_str}_g_{g_str}.hd5'


g_range = (8, 12.5, n_div)
length_range = (1.2, 1.45, n_div)
create_dataset(fname, dt, timesteps, final_resolution, angle_range, velocity_range, g_range, length_range,g_exclude=g_exclude,length_exclude=length_exclude)
print(fname)



g_exclude = (8, 12.5)
length_exclude = (1.2, 1.45)
n_div = 6
angle_range = (30, 170, n_div)
velocity_range = (-2, 2, n_div)
g_range = (12.5, 13.0, n_div)
length_range = (1.45, 1.50, n_div)

num_datapoints = angle_range[2] * velocity_range[2] * g_range[2] * length_range[2]

ang_str = '-'.join([str(a) for a in angle_range[:2]])
vel_str = '-'.join([f'{a:.2f}' for a in velocity_range[:2]])
g_str = '-'.join([f'{a:.2f}' for a in g_range[:2]])
len_str = '-'.join([f'{a:.2f}' for a in length_range[:2]])

fname = f'../data/pixel_pendulum_n_{num_datapoints}_steps_{timesteps}_dt_{dt:.2f}'+\
        f'_angle_{ang_str}_vel_{vel_str}_len_{len_str}_g_{g_str}.hd5'

g_range = (8, 13.0, n_div)
length_range = (1.2, 1.50, n_div)
create_dataset(fname, dt, timesteps, final_resolution, angle_range, velocity_range, g_range, length_range,g_exclude=g_exclude,length_exclude=length_exclude)
print(fname)

# Training set

In [ ]:
n_div = 10
angle_range = (30, 170, n_div)
velocity_range = (-2, 2, n_div)
g_range = (8, 12, n_div)
length_range = (1.2, 1.4, n_div)

num_datapoints = angle_range[2] * velocity_range[2] * g_range[2] * length_range[2]

ang_str = '-'.join([str(a) for a in angle_range[:2]])
vel_str = '-'.join([f'{a:.2f}' for a in velocity_range[:2]])
g_str = '-'.join([f'{a:.2f}' for a in g_range[:2]])
len_str = '-'.join([f'{a:.2f}' for a in length_range[:2]])

fname = f'../data/pixel_pendulum_n_{num_datapoints}_steps_{timesteps}_dt_{dt:.2f}'+\
        f'_angle_{ang_str}_vel_{vel_str}_len_{len_str}_g_{g_str}.hd5'

create_dataset(fname, dt, timesteps, final_resolution, angle_range, velocity_range, g_range, length_range)
print(fname)

Creating 10000 datapoints
0100/10000 finished
0200/10000 finished
0300/10000 finished
0400/10000 finished
0500/10000 finished
0600/10000 finished
0700/10000 finished
0800/10000 finished
0900/10000 finished
1000/10000 finished
1100/10000 finished
1200/10000 finished
1300/10000 finished
1400/10000 finished
1500/10000 finished
1600/10000 finished
1700/10000 finished
1800/10000 finished
1900/10000 finished
2000/10000 finished
2100/10000 finished
2200/10000 finished
2300/10000 finished
2400/10000 finished
2500/10000 finished
2600/10000 finished
2700/10000 finished
2800/10000 finished


# Validation set

In [ ]:
n_div = 6
angle_range = (30, 170, n_div)
velocity_range = (-2, 2, n_div)
g_range = (12, 12.5, n_div)
length_range = (1.4, 1.45, n_div)

num_datapoints = angle_range[2] * velocity_range[2] * g_range[2] * length_range[2]

ang_str = '-'.join([str(a) for a in angle_range[:2]])
vel_str = '-'.join([f'{a:.2f}' for a in velocity_range[:2]])
g_str = '-'.join([f'{a:.2f}' for a in g_range[:2]])
len_str = '-'.join([f'{a:.2f}' for a in length_range[:2]])

fname = f'../data/pixel_pendulum_n_{num_datapoints}_steps_{timesteps}_dt_{dt:.2f}'+\
        f'_angle_{ang_str}_vel_{vel_str}_len_{len_str}_g_{g_str}.hd5'

create_dataset(fname, dt, timesteps, final_resolution, angle_range, velocity_range, g_range, length_range)
print(fname)

# Test set

In [ ]:
n_div = 6
angle_range = (30, 170, n_div)
velocity_range = (-2, 2, n_div)
g_range = (12.5, 13.0, n_div)
length_range = (1.45, 1.50, n_div)

num_datapoints = angle_range[2] * velocity_range[2] * g_range[2] * length_range[2]

ang_str = '-'.join([str(a) for a in angle_range[:2]])
vel_str = '-'.join([f'{a:.2f}' for a in velocity_range[:2]])
g_str = '-'.join([f'{a:.2f}' for a in g_range[:2]])
len_str = '-'.join([f'{a:.2f}' for a in length_range[:2]])

fname = f'../data/pixel_pendulum_n_{num_datapoints}_steps_{timesteps}_dt_{dt:.2f}'+\
        f'_angle_{ang_str}_vel_{vel_str}_len_{len_str}_g_{g_str}.hd5'

create_dataset(fname, dt, timesteps, final_resolution, angle_range, velocity_range, g_range, length_range)
print(fname)

In [ ]:
fg = h5py.File(fname, 'r')
vani=fg['frames'][128]
fig = plt.figure()
ims = []
for i in range(len(vani)):
    ims.append([plt.imshow(vani[i], cmap='gray', vmin=0, vmax=1, aspect='equal', animated=True)])
ani = animation.ArtistAnimation(fig, ims, interval=10, blit=True, repeat_delay=1000)

In [ ]:
plt.imshow(vani[10], cmap='gray', vmin=0, vmax=1, aspect='equal')